In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1705105159375_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#importando bibliotecas
import os
import sys
import pytz
import numpy as np
import datetime
import time
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, udf, lpad, translate, to_timestamp, date_format
from datetime import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg, col, asc, desc, min, avg,max,sum,count,stddev, lit, when, concat

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#funcao freq
### FREQ FUNCTION
def Freq(pTabela,pColuna):

    qtd_total=pTabela.count()

    pTabela.registerTempTable("tab_input")
    frq = spark.sql(
            """
                select
                    {col},
                    count(*) as qtd_absoluto,
                    round(100*(count(*) / {tot}),2) as qtd_percentual
                from
                    tab_input
                group by
                    {col}
                order by
                    {col}
            """.format(col=pColuna, tot=qtd_total))

    qtd=frq.count()
    print('Quantidade de dominios',qtd)
    if qtd > 500:
        frq.show(1000,truncate=False)
        return "Dominio muito granular"

    else:
        frq.show(qtd,truncate=False)
        print("volumetria total:",qtd_total)
        return 'Freq da coluna ' + pColuna;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
agora = datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc = agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark = SparkSession.builder.appName("t_ssp_montly_file").getOrCreate()
sqlContext = SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
### MOD_PER_FILE
raw = "ocorrencia_ssp_2022"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
raw

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'ocorrencia_ssp_2022'

In [8]:
# Endereco Buckets (provisorio)
bucket_control =    "bkt-ssp/Files/05_Control"
bucket_ingestion =  "bkt-ssp/Files/01_IngestionZone"
bucket_raw =        "bkt-ssp/Files/02_RawZone"
bucket_trusted =    "bkt-ssp/Files/03_TrustedZone"
bucket_refined =    "bkt-ssp/Files/04_RefinedZone"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
### MOD_PER_FILE
output_trusted = "v2_ssp_montly_file"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
### MOD_PER_FILE
full_path_trusted = 's3://{bkt}/{file}'.format(bkt=bucket_raw,file=raw)
print(full_path_trusted)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-ssp/Files/02_RawZone/ocorrencia_ssp_2022

In [11]:
raw_00 = spark.read.format("parquet").load(full_path_trusted)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
raw_00.registerTempTable("raw_00")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
for coluna in raw_00.columns:
    print(coluna+',')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Ano,
Janeiro,
Fevereiro,
Marco,
Abril,
Maio,
Junho,
Julho,
Agosto,
Setembro,
Outubro,
Novembro,
Dezembro,
Total,
natureza,
ts_proc,
ts_proc_partition,

In [14]:
tabela = spark.sql(
"""
        SELECT
        
        -- PADRAO PARA TODAS AS TABELAS
        
            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,
        
        -- CAMPOS DO ARQUIVO
        
            CAST(natureza AS STRING) AS natureza,
            CAST(Ano AS INTEGER) AS Ano,
            CAST(Janeiro AS INTEGER) AS Janeiro,
            CAST(Fevereiro AS INTEGER) AS Fevereiro,
            CAST(Marco AS INTEGER) AS Marco,
            CAST(Abril AS INTEGER) AS Abril,
            CAST(Maio AS INTEGER) AS Maio,
            CAST(Junho AS INTEGER) AS Junho,
            CAST(Julho AS INTEGER) AS Julho,
            CAST(Agosto AS INTEGER) AS Agosto,
            CAST(Setembro AS INTEGER) AS Setembro,
            CAST(Outubro AS INTEGER) AS Outubro,
            CAST(Novembro AS INTEGER) AS Novembro,
            CAST(Dezembro AS INTEGER) AS Dezembro
        
        FROM raw_00
"""
.format(pdthproc = dthproc))
tabela.registerTempTable("tabela")
tabela.cache()
tabela.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

23

In [15]:
# Stack Colunas

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
manter = tabela.columns[:4]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
pivotar = tabela.columns[4:]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
manter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ts_proc', 'ts_proc_partition', 'natureza', 'Ano']

In [19]:
pivotar

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Janeiro', 'Fevereiro', 'Marco', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

In [20]:
from pyspark.sql import functions as F

unpivotExp = ", ".join(f"'{col}', {col}" for col in pivotar)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
unpivotExp

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"'Janeiro', Janeiro, 'Fevereiro', Fevereiro, 'Marco', Marco, 'Abril', Abril, 'Maio', Maio, 'Junho', Junho, 'Julho', Julho, 'Agosto', Agosto, 'Setembro', Setembro, 'Outubro', Outubro, 'Novembro', Novembro, 'Dezembro', Dezembro"

In [22]:
tabela_pivotada = tabela.selectExpr(*manter, f"stack({len(pivotar)}, {unpivotExp}) as (mes, valor)"  )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
tabela_pivotada.show(10, truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-----------------+----------------+----+---------+-----+
|ts_proc       |ts_proc_partition|natureza        |Ano |mes      |valor|
+--------------+-----------------+----------------+----+---------+-----+
|20240112215133|20240112215133   |FURTO DE VEÍCULO|2022|Janeiro  |7    |
|20240112215133|20240112215133   |FURTO DE VEÍCULO|2022|Fevereiro|6    |
|20240112215133|20240112215133   |FURTO DE VEÍCULO|2022|Marco    |8    |
|20240112215133|20240112215133   |FURTO DE VEÍCULO|2022|Abril    |7    |
|20240112215133|20240112215133   |FURTO DE VEÍCULO|2022|Maio     |7    |
|20240112215133|20240112215133   |FURTO DE VEÍCULO|2022|Junho    |7    |
|20240112215133|20240112215133   |FURTO DE VEÍCULO|2022|Julho    |8    |
|20240112215133|20240112215133   |FURTO DE VEÍCULO|2022|Agosto   |8    |
|20240112215133|20240112215133   |FURTO DE VEÍCULO|2022|Setembro |8    |
|20240112215133|20240112215133   |FURTO DE VEÍCULO|2022|Outubro  |8    |
+--------------+-----------------+----------------+

In [24]:
tabela_pivotada = tabela_pivotada.withColumn("mes_num",                                
                            when(col('mes') == 'Janeiro', '01')
                            .when(col('mes') == 'Fevereiro', '02')
                            .when(col('mes') == 'Marco', '03')  # Adjust the month numbers accordingly
                            .when(col('mes') == 'Abril', '04')
                            .when(col('mes') == 'Maio', '05')
                            .when(col('mes') == 'Junho', '06')
                            .when(col('mes') == 'Julho', '07')
                            .when(col('mes') == 'Agosto', '08')
                            .when(col('mes') == 'Setembro', '09')
                            .when(col('mes') == 'Outubro', '10')
                            .when(col('mes') == 'Novembro', '11')
                            .when(col('mes') == 'Dezembro', '12')
                            .otherwise(None))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Coluna concat gerada
tabela_pivotada = tabela_pivotada.withColumn("yyyymm", concat('Ano', 'mes_num'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Ordem coluna
new_column_order = ['ts_proc', 'ts_proc_partition', 'yyyymm', 'Ano', 'mes', 'mes_num', 'natureza','valor']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
tabela_pivotada = tabela_pivotada.select(*new_column_order)
tabela_pivotada = tabela_pivotada.orderBy(asc('ts_proc'), asc('ts_proc_partition'), asc('yyyymm'), asc('Ano'), asc('mes_num'), asc('natureza'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#Destino
path_trusted_complete = os.path.join('s3://',bucket_trusted,output_trusted)
print(path_trusted_complete)

tabela_pivotada.coalesce(1).write \
.partitionBy("Ano", "yyyymm", "ts_proc", "ts_proc_partition") \
.parquet(path_trusted_complete, mode='append')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-ssp/Files/03_TrustedZone/v2_ssp_montly_file

In [ ]:
#End